# 程序功能增强

`BrainState` 框架基于[`pygraph` 语法](./pygraph-zh.ipynb)提供了强大的程序功能增强机制，允许用户在基本计算模型的基础上添加额外的功能，如自动微分、批处理、多设备并行等。本教程将详细介绍如何使用这些功能增强特性来优化和扩展你的模型。在阅读本教程之前，建议先阅读[`pygraph` 语法](./pygraph-zh.ipynb)教程。

## 1. 自动微分

自动微分是深度学习中最基础和最重要的功能之一。`BrainState`基于JAX的自动微分系统，提供了[简单直观的API](../apis/augment.rst)来计算梯度。


### 1.1 自动微分语法


`brainstate`提供的自动微分接口需要用户提供所需要求导的``State``集合。基本语法如下：

```python
gradients = brainstate.augment.grad(loss_fn, states)
```

其中，`loss_fn`是损失函数，`states`是需要求导的参数集合。`grad`函数返回一个新的函数，这个函数的输入和`loss_fn`相同，但是返回值是`states`中每个参数的梯度。`grad`针对标量损失函数进行求导，针对别的形式的损失函数，可以将其替换为其它形式的求导函数。目前我们支持的自动微分接口有：

- `brainstate.augment.grad`：标量损失函数的自动微分，使用反向模式自动微分（reverse-mode automatic differentiation）
- `brainstate.augment.vector_grad`：向量函数的自动微分，使用反向模式自动微分
- `brainstate.augment.jacrev`：标量函数的雅可比矩阵，使用反向模式自动微分
- `brainstate.augment.jacfwd`：标量函数的雅可比矩阵，使用前向模式自动微分（forward-mode automatic differentiation）
- `brainstate.augment.jacobian`：标量函数的雅可比矩阵，与`brainstate.augment.jacrev`等价
- `brainstate.augment.hessian`：标量函数的海森矩阵，使用反向模式自动微分
- 更多详细信息请参考[API文档](../apis/augment.rst)


`brainstate`提供的自动微分接口支持返回损失函数值(``return_value=True``)和支持返回辅助数据(auxiliary data, `has_aux=True`)。

当``return_value=True``时，返回值是一个元组，第一个元素是梯度，第二个元素是损失函数值。


```python
gradients, loss = brainstate.augment.grad(loss_fn, states)
```


当``has_aux=True``时，返回值是一个元组，第一个元素是梯度，第二个元素是辅助数据。此时，``loss_fn``需要返回一个元组，第一个元素是损失函数值，第二个元素是辅助数据。

```python
def loss_fn(*args):
    ...
    return loss, aux

gradients, aux = brainstate.augment.grad(loss_fn, states, has_aux=True)
```

当``return_value=True``和``has_aux=True``同时为True时，返回值是一个元组，第一个元素是梯度，第二个元素是损失函数值，第三个元素是辅助数据。

```python
def loss_fn(*args):
    ...
    return loss, aux

gradients, loss, aux = brainstate.augment.grad(loss_fn, states, return_value=True, has_aux=True)
```


### 1.2 基础梯度计算

`brainstate`提供的`grad`和`vector_grad`等函数支持一阶梯度计算。下面是一个简单的例子：


In [1]:
import jax.numpy as jnp

import brainstate as bst

In [2]:
# 创建一个简单的线性层模型
model = bst.nn.Linear(2, 3)

# 准备输入数据
x = jnp.ones((1, 2))
y = jnp.ones((1, 3))


# 定义损失函数
def loss_fn(x, y):
    return jnp.mean((y - model(x)) ** 2)


# 获取模型参数
weights = model.states()

# 计算梯度
grads = bst.augment.grad(loss_fn, weights)(x, y)

# 打印梯度信息
print("Gradients:", grads)


Gradients: {('weight',): {'bias': Array([-0.403517  , -0.25894922, -1.5630271 ], dtype=float32), 'weight': Array([[-0.403517  , -0.25894922, -1.5630271 ],
       [-0.403517  , -0.25894922, -1.5630271 ]], dtype=float32)}}


### 1.3 高阶梯度计算

`BrainState`支持计算高阶导数，这在某些优化任务中非常有用：

In [3]:
# 计算二阶导数
hessian = bst.augment.hessian(loss_fn, weights)(x, y)

# 计算雅可比矩阵
jacobian = bst.augment.jacobian(loss_fn, weights)(x, y)

### 1.4 梯度变换和链式法则

你可以组合多个梯度计算操作：

In [4]:
# 组合多个梯度操作
def composite_grad(fn, params):
    grad_fn = bst.augment.grad(fn, params)
    return lambda *args: bst.augment.grad(grad_fn, params)(*args)

## 2. 批处理增强

批处理(Batching)是深度学习中提高计算效率的关键技术。它通过同时处理多个样本来提高硬件利用率，减少计算开销。``brainstate``支持对[``pygraph``模型](./pygraph-zh.ipynb)进行批处理，用户可以通过简单的API来实现批处理。

### 2.1 批处理创建模型

我们可以通过`brainstate.augment.vmap`函数来将一个模型转换为支持批处理的模型。下面是一个简单的例子：

In [5]:
@bst.augment.vmap
def create_linear(key):
    bst.random.set_key(key)
    return bst.nn.Linear(2, 3)


batch_size = 5

# 创建一个批处理的线性层模型
linears = create_linear(bst.random.split_key(batch_size))

linears

Linear(
  in_size=(2,),
  out_size=(3,),
  w_mask=None,
  weight=ParamState(
    value={'bias': Array([[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]], dtype=float32), 'weight': Array([[[ 0.63507444, -0.24467273,  0.76021475],
            [-0.8693088 , -1.6253686 , -0.35488915]],
    
           [[ 0.42483097, -1.2541455 , -0.05439494],
            [-0.5067744 ,  0.95455235,  1.0039737 ]],
    
           [[ 0.13685398,  0.32665005,  0.27249593],
            [-0.11535253,  1.1269834 , -0.8891422 ]],
    
           [[ 1.7709556 , -2.1601665 ,  0.20920943],
            [-1.5706933 , -0.14618558,  1.7061421 ]],
    
           [[-0.81237787,  0.03658738,  0.9386255 ],
            [-0.27496964, -0.2879541 ,  0.6907252 ]]], dtype=float32)}
  )
)

在这个例子中，我们使用`vmap`函数将一个线性层模型转换为支持批处理的模型。由于模型的初始化基本上都依赖于`brainstate.random`模块随机数，我们需要确保每个模型的随机数种子是不同的。我们可以通过`brainstate.random.split_key`函数来生成不同的随机数种子。`vmap`函数接收随机数种子作为输入，返回一个批处理的模型。

### 2.2 批处理使用模型

我们可以按照某个维度对模型的状态变量进行批处理。下面是一个简单的例子：


In [6]:
# 创建批处理数据
x_batch = jnp.ones((batch_size, 2))
y_batch = jnp.ones((batch_size, 3))


# 对每一个批次计算损失
@bst.augment.vmap(in_axes=(0, 0))
def eval(model, x):
    return model(x)


# 批处理版本的损失函数
def batch_loss_fn(x_batch, y_batch):
    predictions = eval(linears, x_batch)
    return jnp.mean((y_batch - predictions) ** 2)


# 计算批处理梯度
weights = linears.states(bst.ParamState)
batch_grads = bst.augment.grad(batch_loss_fn, weights)(x_batch, y_batch)

batch_grads

{('weight',): {'bias': Array([[-0.16456458, -0.3826722 , -0.07928993],
         [-0.14425914, -0.1732791 , -0.00672283],
         [-0.13046649,  0.06048446, -0.21555285],
         [-0.1066317 , -0.44084698,  0.12204687],
         [-0.278313  , -0.16684891,  0.08391342]], dtype=float32),
  'weight': Array([[[-0.16456458, -0.3826722 , -0.07928993],
          [-0.16456458, -0.3826722 , -0.07928993]],
  
         [[-0.14425914, -0.1732791 , -0.00672283],
          [-0.14425914, -0.1732791 , -0.00672283]],
  
         [[-0.13046649,  0.06048446, -0.21555285],
          [-0.13046649,  0.06048446, -0.21555285]],
  
         [[-0.1066317 , -0.44084698,  0.12204687],
          [-0.1066317 , -0.44084698,  0.12204687]],
  
         [[-0.278313  , -0.16684891,  0.08391342],
          [-0.278313  , -0.16684891,  0.08391342]]], dtype=float32)}}

在这个例子中，我们使用`vmap`函数对每个批次的输入数据进行了批处理。`in_axes`参数指定了哪些维度是批处理的维度。在这个例子中，我们对第一个维度进行了批处理。`eval`函数接收一个批处理的模型和一个批处理的输入数据，返回一个批处理的输出数据。`batch_loss_fn`函数接收一个批处理的输入数据和一个批处理的输出数据，返回一个标量损失值。`grad`函数接收一个批处理的损失函数和一个批处理的模型参数，返回一个批处理的梯度。

这个例子中，所有``State``的值被read，如果存在每个``State``的值被write，那么``vmap``函数会自动把这些批处理后的``State``值拉取更新出来。

In [7]:
class Foo(bst.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = bst.ParamState(jnp.arange(4))
        self.b = bst.ShortTermState(jnp.arange(4))

    def __call__(self):
        self.b.value = self.a.value * self.b.value


@bst.augment.vmap
def mul(foo):
    foo()


foo = Foo()
mul(foo)

foo.b.value

Array([0, 1, 4, 9], dtype=int32)

在上面的例子中，我们定义了一个简单的模型`Foo`，它包含一个`ParamState`和一个`ShortTermState`。我们使用`vmap`函数将这个模型转换为支持批处理的模型。在`mul`函数中，我们对这个批处理的模型进行了调用。在这个例子中，`vmap`函数会自动识别`a`和`b`的值是批处理的，`vmap`函数会自动拉取这些批处理后的值。

### 3. 指定批处理``State``

在建模过程中，我们经常遇到这样的情况：我们需要对某些``State``进行批处理，而对另一些``State``不进行批处理。`brainstate`提供了``brainstate.augment.StateAxes``类来指定哪些``State``需要进行批处理。``StateAxes``可以用于指定``vmap``的`in_axes`参数和`out_axes`参数。下面是一个简单的例子。


In [8]:
import brainunit as u


class LIFNet(bst.nn.Module):
    def __init__(self, nin, nout):
        super().__init__()

        self.i2r = bst.nn.Linear(nin, nout)
        self.lif = bst.nn.LIF(nout)

    def update(self, x):
        r = self.i2r(x)
        return self.lif(r * u.mA)

在这个例子中，我们定义了一个简单的LIF神经元模型。这个模型包含一个线性层和一个LIF神经元。

In [9]:
n_in = 2
n_out = 3
batch_size = 5

net = LIFNet(n_in, n_out)


@bst.augment.vmap(out_axes=bst.augment.StateAxes({'new': 0, ...: None}))
def init_net(key):
    bst.random.set_key(key)

    # 初始化一个batch的模型状态变量
    with bst.catch_new_states('new'):
        bst.nn.init_all_states(net)

    # 返回一个batch的模型
    return net


net = init_net(bst.random.split_key(batch_size))

在这个例子中，我们使用`StateAxes`类来指定哪些``State``需要进行批处理。我们首先使用``brainstate.catch_new_states``catch所有新初始化的``State``，并赋予他们一个相同的tag (``tag='new'``)。在``vmap``函数中，我们使用``StateAxes``来指定所有``tag='new'``的``State``需要进行批处理。这样，我们就可以对新初始化的``State``进行批处理操作了。

In [10]:
net.lif.V.value

ArrayImpl([[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]], dtype=float32) * mvolt

In [11]:
@bst.augment.vmap(in_axes=(bst.augment.StateAxes({'new': 0, ...: None}), 0))
def batch_run(model, x):
    with bst.environ.context(dt=0.1 * u.ms):
        o = model.update(x)
    return o


xs = bst.random.rand(batch_size, 2) < 0.5

r = batch_run(net, xs)

In [12]:
net.lif.V.value

ArrayImpl([[-0.01608765, -0.02008961,  0.03482756],
           [-0.01781268, -0.04330517,  0.07538113],
           [-0.01781268, -0.04330517,  0.07538113],
           [-0.01608765, -0.02008961,  0.03482756],
           [ 0.        ,  0.        ,  0.        ]], dtype=float32) * mvolt

在这个例子中，我们使用`vmap`函数对上面我们批初始化的模型进行了调用。在`run_model`函数中，我们使用`StateAxes`来指定所有``tag='new'``的``State``需要进行批处理，其它的``State``实例不需要进行批处理。

通过上面的示例，我们可以看到，即便我们的模型定义是单批次的，但我们依然可以通过灵活地调用`vmap`函数对模型进行批处理。这样，我们就可以在模型定义中不用考虑批处理的问题，而在模型调用时再决定是否进行批处理。

## 3. 多设备并行计算


不仅仅是批处理，`brainstate`还支持多设备并行计算。我们可以通过`brainstate.augment.pmap`函数将一个模型转换为支持多设备并行计算的模型。

`brainstate.augment.pmap`增强函数的用法基本上跟`brainstate.augment.vmap`函数是一样的，只是`pmap`函数会将模型转换为支持多设备并行计算的模型，而`vmap`函数只是但设备上不同线程上的并行计算。

## 4. 组合使用功能增强

在实际应用中，我们常常需要组合使用多种功能增强。``brainstate``中各种功能增强函数和编译函数之间是可以互相组合使用的。下面是一个简单的例子：


In [13]:
ys = bst.random.rand(batch_size, 3)

def loss_fn(x, y):
    return jnp.mean((y - batch_run(net, x)) ** 2)


weights = net.states(bst.ParamState)
opt = bst.optim.Adam(1e-3)
opt.register_trainable_weights(weights)


@bst.compile.jit
def batch_train(xs, ys):
    grads, l = bst.augment.grad(loss_fn, weights, return_value=True)(xs, ys)
    opt.update(grads)
    return l
    
    
l = batch_train(xs, ys)
l

Array(0.4479962, dtype=float32)

## 5. 性能优化建议

使用功能增强时，需要注意以下几点以获得最佳性能：

1. **合理的批大小**：根据设备内存和计算能力选择适当的批大小
2. **梯度累积**：当批大小受限时，考虑使用梯度累积
3. **缓存编译**：重复使用已编译的函数以减少编译开销
4. **内存管理**：使用`jax.device_get()`及时释放设备内存


## 6. 调试技巧

在使用功能增强时，调试是一个重要话题。`brainstate`完全支持jax中提供的调试工具，如`jax.debug`中的`print`函数。下面是一个简单的例子：

```python
# 使用jax.debug.print进行调试
@bst.compile.jit
def batch_train(xs, ys):
    grads, l = bst.augment.grad(loss_fn, weights, return_value=True)(xs, ys)
    jax.debug.print("Gradients: {g}", g=grads)
    opt.update(grads)
    return l

```

详细用法用户可以参考[JAX调试文档](https://jax.readthedocs.io/en/latest/debugging/index.html)。


## 总结

基于`pygraph`的程序功能增强是`BrainState`框架的核心特性之一，通过合理使用这些功能，可以显著提升模型的推理、训练效率和性能。本教程涵盖了主要的功能增强特性及其使用方法，为进一步探索和应用这些特性提供了基础。